In [1]:
using JuliaDB
using IndexedTables
using Dagger

using VCFTool

In [7]:
input_dir = "../input/"

vcf_738_1000_file_path = joinpath(input_dir, "738_variants_1000.vcf.gz")

vcf_file_path_to_use = vcf_738_1000_file_path

"../input/738_variants_1000.vcf.gz"

In [8]:
vcf_table = make_vcf_indexedtable(vcf_file_path_to_use)

Table with 999 rows, 10 columns:
Columns:
#   colname  type
───────────────────
1   CHROM    Any
2   POS      Any
3   ID       String
4   REF      String
5   ALT      String
6   QUAL     String
7   FILTER   String
8   INFO     String
9   FORMAT   String
10  GERM     String

In [4]:
Dagger.save(vcf_table, "../input/738_variants_1000.vcf.dagger")

Table with 999 rows, 10 columns:
Columns:
#   colname  type
───────────────────
1   CHROM    Any
2   POS      Any
3   ID       String
4   REF      String
5   ALT      String
6   QUAL     String
7   FILTER   String
8   INFO     String
9   FORMAT   String
10  GERM     String

In [6]:
using Dagger

vcf_table_dagger = Dagger.load("../input/738_variants_1000.vcf.dagger")

Table with 999 rows, 10 columns:
Columns:
#   colname  type
───────────────────
1   CHROM    Any
2   POS      Any
3   ID       String
4   REF      String
5   ALT      String
6   QUAL     String
7   FILTER   String
8   INFO     String
9   FORMAT   String
10  GERM     String

# Dagger load v.s. load from file

Here we compare how long it takes to access a particular variant and a chromosomal region in a regular JuliaDB IndexedTable that was loaded directly from a file versus an index file that was created via `Dagger.save` and loaded here via `Dagger.load`. Using Dagger to save and load a VCF adds another step and dependency to data prep, so this speed test will tell if its worth it.

In [13]:
benchmark_variant = [1, 13868]

benchmark_region = [1, 10000, 100000]

3-element Array{Int64,1}:
      1
  10000
 100000

#### IndexedTable from file

In [14]:
using Dates

start_time = now()

filter(i -> (i.CHROM == benchmark_variant[1]) && (i.POS == benchmark_variant[2]), vcf_table)

end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")


Took 3 seconds, 985 milliseconds.



#### Dagger file

In [15]:
using Dates

start_time = now()

filter(i -> (i.CHROM == benchmark_variant[1]) && (i.POS == benchmark_variant[2]), vcf_table_dagger)

end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")


Took 104 milliseconds.



# Access VCF data from NDSParse v.s. IndexedTable

Indexed Table should win.